In [1]:

import scipy as sp 
from scipy import signal
#from signal import butter
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from pathlib import Path
#import librosa
import sklearn
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from keras import layers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Reshape , Dropout
from keras.regularizers import l1
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split

#importo funzioni di pre-processing
from PreProcessing.processing import *
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
  #raise SystemError('GPU device not found')    
else:
  print('I Found a GPU! at: {}'.format(device_name))
plt.style.use('seaborn')

GPU device not found


C:\Users\david\AppData\Local\Temp\ipykernel_21944\3230636011.py:29: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [3]:
tf.config.list_physical_devices('GPU')
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17959731461096736053
xla_global_id: -1
]


### RowPower signal preprocessing
here we apply the same preprocessing that the system does in real time before fedding the data into the model. This allow us to train the model with back up datasets. 

In [2]:
#RAW path

raw_data = {
    'arpeggio_raw' : pd.read_csv("C:\\Users\david\OneDrive - Politecnico di Milano\Documenti\GItDesktop\MAE_Thesis\\firstPrototype\secondAcquisition\\1 Arpeggio\Arpeggio.csv",sep=';'),
    'strumming_raw': pd.read_csv("C:\\Users\david\OneDrive - Politecnico di Milano\Documenti\GItDesktop\MAE_Thesis\\firstPrototype\secondAcquisition\\2 Strumming\Strumming.csv",sep=';'),
    'bending_raw' :  pd.read_csv("C:\\Users\david\OneDrive - Politecnico di Milano\Documenti\GItDesktop\MAE_Thesis\\firstPrototype\secondAcquisition\\3 Bending\Bending.csv",sep=';'),
    'pullOffHammerOn_raw': pd.read_csv("C:\\Users\david\OneDrive - Politecnico di Milano\Documenti\GItDesktop\MAE_Thesis\\firstPrototype\secondAcquisition\\4 PullOffHammerOn\PullOffHammerOn.csv",sep=';'),
    'tapping_raw': pd.read_csv("C:\\Users\david\OneDrive - Politecnico di Milano\Documenti\GItDesktop\MAE_Thesis\\firstPrototype\secondAcquisition\\5 Tapping\\tapping.csv",sep=';'),
    'strongPick_raw':  pd.read_csv("C:\\Users\david\OneDrive - Politecnico di Milano\Documenti\GItDesktop\MAE_Thesis\\firstPrototype\secondAcquisition\\6 StrongPick\\strongPick.csv",sep=';'),
    'doublePick_raw' : pd.read_csv("C:\\Users\david\OneDrive - Politecnico di Milano\Documenti\GItDesktop\MAE_Thesis\\firstPrototype\secondAcquisition\\7 DoublePick\\doublePick.csv",sep=';') ,
}
scaler = MinMaxScaler()

for i in raw_data:
  raw_data[i]= raw_data[i].iloc[5000:35000,3:].values # excluding the first and the last 5 second of samples and trasmorming it it np.array 
  raw_data[i] = scaler.fit_transform(raw_data[i])#scales the data from (0,1)
  raw_data[i]= np.reshape(raw_data[i],(raw_data[i].shape[0], 1, raw_data[i].shape[1]))
print('The lenght and shapes of my training datasets = {}, {}'.format(len(raw_data), [raw_data[key].shape for key in raw_data]))
print(raw_data['arpeggio_raw'])

#remanber to set the same keys name of the trainig dictonary and the target dictionary
#doig so you can apply the same dimensionality[0] of the training set to the first dimension of the testing set 

#Target Set
target_dict = {
    'arpeggio_raw': np.array([0.5,0.3, 0.2, 0.3, 0.5]),
    'strumming_raw': np.array([0.6,0.2, 0.8, 0.7, 0.3]),
    'bending_raw': np.array([0.2,0.1, 0.1, 0.5, 0.2]),
    'strongPick_raw': np.array([0.4,0.5, 0.3, 0.4, 0.1]),
    'tapping_raw': np.array([0.3,0.2, 0.6, 0.6, 0.7]),
    'pullOffHammerOn_raw': np.array([0.7,0.7, 0.8, 0.2, 0.4]),
    'doublePick_raw': np.array([0.8,0.4, 0.9, 0.1, 0.8]),
}
#populate the dictionaty with the right shapes before using them as target set
# i have to create a target set with 8 rows and 5 columns 

for i in target_dict:  
  target_dict[i]= np.tile(target_dict[i], (raw_data[i].shape[0],1))

print("The Shape of the Target set: ",[i.shape for i in target_dict.values()])

#arpeggio_row= pd.read_csv(arpeggio_path_raw,sep= ';')
#arpeggio_row = arpeggio_row.iloc[:,3:]

#first I hae to remouve the first 3 column that are not data but timestemp ID and Sequence
#print(arpeggio_row)



The lenght and shapes of my training datasets = 7, [(30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8)]
[[[0.57640807 0.57159955 0.64360317 ... 0.3120852  0.45602244 0.43430842]]

 [[0.55510371 0.55204205 0.65485674 ... 0.3187992  0.45584321 0.44931653]]

 [[0.56572256 0.5760748  0.66004097 ... 0.34812471 0.47683057 0.42754662]]

 ...

 [[0.54029447 0.56749725 0.72417368 ... 0.33624016 0.63961251 0.45171062]]

 [[0.58314127 0.59708936 0.66282275 ... 0.36139836 0.6746243  0.41489827]]

 [[0.58152226 0.57347291 0.55612877 ... 0.36078098 0.71554158 0.4345401 ]]]
The Shape of the Target set:  [(30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5)]


Now i have imported and shaeped the dataset i need to preprocess them wiht a butter band pass filter. It's good to plot your training set if you can.

In [22]:
#plt.figure(figsize=(12,8))
#plt.plot(raw_data['arpeggio_raw'])

In [ ]:
sos = signal.butter(5, [30,300], 'bandpass', fs=1000, output='sos') #second order section filter band pass

#applying band pass filter 
for i in raw_data:
    raw_data[i]= signal.sosfilt(sos, raw_data[i].astype(np.float32))

print('Training dataset lenght {} and shape {}'.format(len(raw_data), [raw_data[key].shape for key in raw_data]))

print(raw_data['arpeggio_raw'])

## Applying the RMS 
Now we apply a window rms to obtain the same number of sample but in a RMS form.

In [4]:
#prove di funzionamento per popolare il 
df_toLSTM= pd.DataFrame(columns=['g00', 'g01', 'g02', 'g03','g10', 'g11', 'g12', 'g13'])
g00= np.random.uniform(low=-10000, high=10000, size=500)
df_toLSTM['g00'] = g00

print(df_toLSTM)

df_toLSTM['g00'] = np.random.uniform(low=-10000, high=10000, size=500)
print(df_toLSTM)

             g00  g01  g02  g03  g10  g11  g12  g13
0   -5304.710925  NaN  NaN  NaN  NaN  NaN  NaN  NaN
1     425.610817  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2     101.243193  NaN  NaN  NaN  NaN  NaN  NaN  NaN
3    8925.926500  NaN  NaN  NaN  NaN  NaN  NaN  NaN
4    6448.940477  NaN  NaN  NaN  NaN  NaN  NaN  NaN
..           ...  ...  ...  ...  ...  ...  ...  ...
495 -6220.534876  NaN  NaN  NaN  NaN  NaN  NaN  NaN
496 -1964.807550  NaN  NaN  NaN  NaN  NaN  NaN  NaN
497  8493.520523  NaN  NaN  NaN  NaN  NaN  NaN  NaN
498  2494.440886  NaN  NaN  NaN  NaN  NaN  NaN  NaN
499  3339.880792  NaN  NaN  NaN  NaN  NaN  NaN  NaN

[500 rows x 8 columns]
             g00  g01  g02  g03  g10  g11  g12  g13
0   -4914.564746  NaN  NaN  NaN  NaN  NaN  NaN  NaN
1   -8827.948709  NaN  NaN  NaN  NaN  NaN  NaN  NaN
2     -47.334185  NaN  NaN  NaN  NaN  NaN  NaN  NaN
3   -3088.538524  NaN  NaN  NaN  NaN  NaN  NaN  NaN
4    8838.718929  NaN  NaN  NaN  NaN  NaN  NaN  NaN
..           ...  ...  ...  ...  ...  ..

In [4]:

random_data = {
    'g00': np.random.uniform(low=-10000, high=10000, size=500),
    'g01': np.random.uniform(low=-10000, high=10000, size=500),
    'g02': np.random.uniform(low=-10000, high=10000, size=500),
    'g03': np.random.uniform(low=-10000, high=10000, size=500),
    'g10': np.random.uniform(low=-10000, high=10000, size=500),
    'g11': np.random.uniform(low=-10000, high=10000, size=500),
    'g12': np.random.uniform(low=-10000, high=10000, size=500),
    'g13': np.random.uniform(low=-10000, high=10000, size=500)
}

df_toLSTM_1= pd.DataFrame(random_data)
'''
df_toLSTM_1= df_toLSTM_1.values
#if the nuber of initial sample if divisible by the group size you can create groups of group_size only using reshape(-1, group_size, 8)
df_toLSTM_1= df_toLSTM_1.reshape(-1,25,8)
print(df_toLSTM_1.shape)ù
'''
# Reshape the DataFrame into the desired shape
#reshaped_df = df_toLSTM_1.values.reshape(-1, 32, 8)
#df_toLSTM_1= scaler.fit_transform(df_toLSTM_1)
#scaler= MinMaxScaler()
#for i in random_data:
#    random_data[i]= scaler.fit_transform(random_data[i])
#df_toLSTM_1 = df_toLSTM_1.values

print('\n The new dataset size ', '\n', df_toLSTM_1.shape)
#print('\n The new normalized dataset  ', '\n', df_toLSTM_1)


 The new dataset size  
 (500, 8)


NameError: name 'RNN_input_preprocessing' is not defined

Ok ho definito un dataset di prova. ora devo solo provare a darlo in pasto al modello per vedere se me lo predice bene.

Whit the input_preprocessing function I pack my data before feeding them to the model: I adacts my inputs dataframe to have the model imput shape, I covert the row data into a rms and I filter them wiht a notch and a band pass filter.

In [8]:

def RNN_input_preprocessing(raw_dataframe, fs= 1000): 
    '''
    creates a dataframe containing filtered Emg signals (Df_Emg_Filtered), a dataframe containing smoothed signals (Df_Savitzky) a dataframe containing RMS signals (Df_RMS) and a datafreme containing MNF signals (Df_MNF)
    Args:
        only_DF_RMS(bool): if True only Df_RMS will be compute otherwise Df_Emg_Filtered, Df_RMS, Df_Savitzky and Df_RMS will be computed 
    '''
    #Defining the band pass parameters and the windows size
    lowcut = 30.0 #inizializzazione frequenza taglia basso
    highcut = 300.0  #inizializzazione frequenza taglia alto (verrà comunque detrminata automaticamente all'apertura del file EMG)
    order=5 #ordine del filtro di butterworth
    rms_window_size=125 #numero di campioni su cui effettuare la convoluzione per il calcolo del RMS

    #CALCOLO DF SEGNALI FILTRATI
    #casting to int before applying the bandpas, othetwise it rainses ad exeption
    #for i in range(len(raw_dataframe.columns[:-1])):
    #   raw_dataframe[raw_dataframe.columns[i]] = raw_dataframe[raw_dataframe.columns[i]].astype('int64')

    
    #applicazione filtro passa panda e di notch su ogni colonna (canale) del dataframe Raw
    #raw_dataframe= raw_dataframe.fillna(0).astype(int)
    #now we apply the notch filter feeding it with the prefiltered bandapassed stream 
    df_Emg_Filtered= raw_dataframe.apply(lambda x: Implement_Notch_Filter(fs,1,50, None, 3, 'butter', 
                                        butter_bandpass_filter(x,lowcut,highcut, fs, order=order)))

    
    #CALCOLO DF SEGNALI RMS
    def df_rms(vect):
        rect=abs(vect.fillna(0).values)
        for i in range(25):
            rect[i]=int(1)
        return window_rms(rect,rms_window_size)

    #applicazione della funzione df_rms su ogni colonna (canale) del dataframe contente i segnali filtrati
    #apply the RMS to the band passed dataframe
    df_RMS= df_Emg_Filtered.apply(lambda x: df_rms(x))
    df_RMS= df_RMS.round(2)                                                                                                                                                                                                                                                                                                                                                     

    
    #tempo del segnale rms è uguale a quello emg solo tagliato all'inizio e alla fine per via della convoluzione 'valid'
    #il primo campione del RMS è stato assegnato a metà della prima finestra
    #tempo_rms= raw_dataframe['TIMESTAMP'][int(rms_window_size/2):-int(rms_window_size/2)+1] 
    
    print('df_RMS with initial shape ', df_RMS.shape) # '\n', df_RMS, '\n', type(df_RMS))

    #scaling the dataset 
    #raw_dataframe= scaler.fit_transform(raw_dataframe) # it return an np array 
    #df_RMS= df_RMS.values

    #print('the shape of the df array: ', df_RMS.shape)
    #scaling the value from (0,1)ù
    scaler = MinMaxScaler()
    df_RMS= scaler.fit_transform(df_RMS)

    #df_RMS[i]= df_RMS[i].values

    sequence_length = 32
    # Print the shape of the raw data before reshaping
    #print("Raw Data Shape:", df_RMS.shape[0])

    # Calculate the number of groups
    num_groups = df_RMS.shape[0] // sequence_length
    # Print the number of groups
    #print("Number of Groups:", num_groups)
    
    casting_index= num_groups * sequence_length
    # Calculate the number of remaining elements
    #num_remaining = df_RMS[i].shape[0] % sequence_length
    
    # Reshape the input data into a 3D matrix
    input_sequences = np.reshape(df_RMS[:casting_index], (num_groups, sequence_length, df_RMS.shape[1]))
    df_RMS = input_sequences     


    # The shape of 'input_sequences' will be (num_groups + 1, sequence_length, num_features)
    print('The final shape:' , df_RMS.shape)

    return df_RMS
df_arpeggio_RMS = RNN_input_preprocessing(df_toLSTM_1)




df_RMS with initial shape  (376, 8)
The final shape: (11, 32, 8)


### Defining the Training and Target set
 
To define the target set, it is necessary to ensure that its first dimension matches that of the training set (the number of rows). We have a training set of shape(num_samples,1, 8), it means with 8 features each wiht dimension 1. We want to have a 5 floats outputs so we need to define a target set of num_sample rows and 5 targets ( 5 output features). 

The idea is create two dictionary: the training and the target. I need them wiht the same keys name to guarantee that when I load a new acquisition of one of the training dict element, i will concatenate the last and the dimension of the corresponding target dict will increase the number of samples accordingly. 

In [6]:
#completare con valori sensati

#Training set 
pd_dataframe= {
    'arpeggio': pd.read_csv('csv_pd_dataframe\df_arpeggio_rms'),
    'strumming': pd.read_csv('csv_pd_dataframe/df_strumming_rms'),
    'bending': pd.read_csv('csv_pd_dataframe/df_bending_rms'),
    'double pick':pd.read_csv('csv_pd_dataframe/df_doublePick_rms'),
    'strong pick':pd.read_csv('csv_pd_dataframe/df_strongPick_rms'),
    'tapping':pd.read_csv('csv_pd_dataframe/df_tapping_rms'),
    'pullOff Hammer on': pd.read_csv('csv_pd_dataframe/df_pullOffHammerOn_rms')
}
#reshaping all the pd dataset to be (samples,time steps, features) that in our case is (30000, 1, 8) shape 
for i in pd_dataframe:
  pd_dataframe[i]= pd_dataframe[i].iloc[5000:35000].values # excluding the first and the last 5 second of samples and trasmorming it it np.array 
  pd_dataframe[i]= np.reshape(pd_dataframe[i],(pd_dataframe[i].shape[0], 1, pd_dataframe[i].shape[1]))
print('The lenght and shapes of my training datasets = {}, {}'.format(len(pd_dataframe), [pd_dataframe[key].shape for key in pd_dataframe]))

#Printing some statistics 
for i in pd_dataframe.keys():
  print('mean and variance of', i ,' = {:.3f}, {:.3f}'.format(np.mean(pd_dataframe[i]), np.var(pd_dataframe[i])))

#Target Set
target_dict = {
    'arpeggio': np.array([0.5,0.3, 0.2, 0.3, 0.5]),
    'strumming': np.array([0.6,0.2, 0.8, 0.7, 0.3]),
    'bending': np.array([0.2,0.1, 0.1, 0.5, 0.2]),
    'strongPick': np.array([0.4,0.5, 0.3, 0.4, 0.1]),
    'tapping': np.array([0.3,0.2, 0.6, 0.6, 0.7]),
    'pullOffHammerOn': np.array([0.7,0.7, 0.8, 0.2, 0.4]),
    'doublePick': np.array([0.8,0.4, 0.9, 0.1, 0.8]),
}
#populate the dictionaty with the same sample as the training set before using them as target set, 
# i have to create a target set with 30000 rows and 5 columns (samples, outputs)
for i in target_dict:  
  target_dict[i]= np.tile(target_dict[i], (30000,1))

print("The Shape of the Target set: ",[i.shape for i in target_dict.values()])

The lenght and shapes of my training datasets = 7, [(30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8), (30000, 1, 8)]
mean and variance of arpeggio  = 2236.176, 1305701.184
mean and variance of strumming  = 4943.854, 26904060.131
mean and variance of bending  = 3656.116, 151639304.146
mean and variance of double pick  = 2294.655, 2121090.473
mean and variance of strong pick  = 2088.049, 2635636.172
mean and variance of tapping  = 2379.778, 2249018.599
mean and variance of pullOff Hammer on  = 2526.514, 6551681.842
The Shape of the Target set:  [(30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5), (30000, 5)]


## Training dataset packing
Here I pack my input data to feed them to the Model during the training process. Furthemore we split the Training dataset in two part, one for the training process and one for the evalution, called Test set.

In [9]:

concatenated_training_datasets= np.concatenate(([pd_dataframe[key] for key in pd_dataframe.keys()]),axis=0)
#print(x_train.shape)
concatenated_target_datasets= np.concatenate(([target_dict[key] for key in target_dict.keys()]), axis= 0)
#print(y_train.shape)
#splitting the dataset into a training and testing dataset



X_train, X_test, Y_train, Y_test = train_test_split(concatenated_training_datasets, concatenated_target_datasets, test_size=0.2, random_state=42)
print('The traing set shape {} , the test set shape {}, the tartet set shape {} , the target testing set shape {}'.format(X_train.shape ,X_test.shape, Y_train.shape, Y_test.shape))

The traing set shape (168000, 1, 8) , the test set shape (42000, 1, 8), the tartet set shape (168000, 5) , the target testing set shape (42000, 5)


## LSTM Sequential Network
Un idea per migliorare le perfomance sarebbe provare a normalizzare l'input da 0,1 0 da -1 a 1 e provare ad allenare la rete con i dati row invece che con l'RMS. 



In [10]:
model = Sequential()
#By setting the input shape to (8, 1), you are specifying that each sample consists of 8 time steps, each time step consisting of a single feature
# Add an input dense layer with 8 neurons, one for each column in the input dataset

# Add an LSTM layer with 64 units
model.add(LSTM(64, input_shape= (1,8),return_sequences=True, kernel_regularizer=l1(0.1))) 
# when you want to stack multiple LSTM 
#you have to set the return_sequences=True to not have shape problems. 
#return_sequences: Boolean. Whether to return the last output in the output sequence, or the full sequence. Default: False

model.add(Dropout(0.3))  # add a dropout layer with dropout rate of 0.2
model.add(LSTM(64))
# Add a dense output layer with 5 units and linear activation
model.add(Dense(5, activation='linear'))  

print('The output shape: ',model.output_shape)
model.summary()

The output shape:  (None, 5)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 64)             18688     
                                                                 
 dropout (Dropout)           (None, 1, 64)             0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 5)                 325       
                                                                 
Total params: 52,037
Trainable params: 52,037
Non-trainable params: 0
_________________________________________________________________


## Compiling the Model
To compile the network (before training it) we have to choose the Optimizer and the loss function: 

The optimizer is the algorithm used to update the weights of the model during training, and 'adam' is a popular optimizer that adapts the learning rate of each weight based on the first and second moments of the gradients.

The loss function is used to measure how well the model is performing on the training data and guides the optimization process. 'mse' stands for mean squared error, which is a common loss function used for regression problems where the goal is to minimize the difference between the predicted and actual values.

Adam optimization is a stochastic gradient descent method that is based on adaptive estimation of first-order and second-order moments.

Learning rate: The learning rate determines how much the model weights are updated during training. A larger learning rate can lead to faster convergence, but can also cause the model to overshoot the minimum.


In [ ]:
#Compiling the model

myOptimizer = keras.optimizers.Adam(lr=0.01)
model.compile(optimizer= myOptimizer, loss='mse', metrics=['mse'])


## Training the Model

In [ ]:
history = model.fit(X_train,Y_train, validation_split=0.25, batch_size= 64, epochs=75)


The validation_split argument in Keras fit method specifies the fraction of the training data to be used as validation data. The model will not be trained on the validation data, but rather the validation data will be used to evaluate the model's training performance over the epochs on data it has not seen before.

For example, if validation_split=0.2, then 20% of the training data will be used as validation data and the remaining 80% will be used as the training data. The validation data will be used to evaluate the model's performance after each epoch during training. The validation set is a portion of the training set to evaluate the training process, instead to evaluate the network we have to use a different dataset called testing dataset.

Batch size: The batch size determines the number of training examples that are used in each training iteration. Larger batch size can lead to faster training, but can also lead to slower convergence and poorer generalization.

The number of LSTM layers: Increasing the number of layers can help capture more complex relationships in the data, but can also increase the risk of overfitting.

The number of LSTM units: Increasing the number of neurons can help the model capture more complex patterns in the data, but can also increase the risk of overfitting.

Learning rate: The learning rate determines how much the model weights are updated during training. A larger learning rate can lead to faster convergence, but can also cause the model to overshoot the minimum.


### Plotting the perormance

We can track the training accurancy to avoid overfitting by plotting the loss function and the MSE for each epoch. The MSE is a common metric to evaluate regession task. The lower the loss, the better the model performance. The purpose of the loss function is to evaluate how well the model is learning the patterns in the data and adjusting its weights and biases to make accurate predictions. So, when you train a model, you try to minimize the loss function by adjusting the model's parameters. In other evaluation metrics, such as accuracy or F1 score, the higher the value, the better the model performance.
Ploting the training history:

In [ ]:
plt.figure(figsize=(16,5))
plt.title('arpeggio')
plt.subplot(1,2,1)

# summarize history for accuracy
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.title('Model MSE')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

plt.subplot(1,2,2)
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')

## Model Evaluation 
Testing the model whit a testing set computing the loss function for an unseen dataset. 
The testing dataset have to be useen data but whith a trained target set, you can use a dataset belonging to a target that you did not train. In the training the network see the label, in the tasting set it see the label only at the end, but shold be data from a studied label otherwise it has no sense. 
it has to be data belonging to one of the category which you use during the training but unseen during the training stage.
you have to split the training dataset to obtai a testing dataset. 


In [9]:
## Model evaluation 
model.evaluate(X_test,Y_test)

NameError: name 'X_test' is not defined

## Prediction of new data
Now we have to actualy try the regression, we could use the method model.predict

In [27]:
# Load the model
model = load_model('sEMG_regression.h5')

# Reshape the data to match the input shape of the model

#new_data= pd_dataframe['bending'][2]
new_data= df_arpeggio_RMS.values

new_data= np.reshape(new_data,(new_data.shape[0],1, new_data.shape[1]))
print(new_data.shape) 
#new_data = new_data.reshape(1, -1)

# Make a prediction using the model
prediction = model.predict(new_data)

print(prediction, prediction.shape)

(376, 1, 8)
12/12 [==============================] - 1s 2ms/step
[[0.57763714 0.46410373 0.7094731  0.66726536 0.13209713]
 [0.57763714 0.46410373 0.7094731  0.66726536 0.13209713]
 [0.57763714 0.46410373 0.7094731  0.66726536 0.13209713]
 ...
 [0.5999903  0.2999681  0.79445815 0.6954058  0.30008966]
 [0.5999903  0.2999681  0.79445815 0.6954058  0.30008966]
 [0.5999903  0.2999681  0.79445815 0.6954058  0.30008966]] (376, 5)


## Exporting the model 
Now we want to save the model to use it in RawPower

In [ ]:
# Save the model
model.save('sEMG_regression.h5')

Then we could load again the model from everywhere else, be sure to import the same libraries imported at the beginning.

In [5]:
# Load the model
model = load_model('sEMG_regression.h5')